In [1]:
from typing import List, Tuple, Any
from datetime import datetime

In [44]:

def get_unix(date, time):
    format_date = "%d/%m/%Y"
    format_time = "%H:%M:%S.%f000"
    date_obj = datetime.strptime(date, format_date)
    time_obj = datetime.strptime(time, format_time)
    almost = datetime.combine(date_obj.date(), time_obj.time())
    return (almost.timestamp())

class Tick:
  def __init__(self,line: List[int]) -> None:
    splitted_line = line.split(',')
    self.date = splitted_line[0]
    self.time = splitted_line[1]
    self.bid = splitted_line[2]
    self.ask = splitted_line[4]
    #self.bid_volume = splitted_line[3]
    #self.ask_volume = splitted_line[5]
    self.unix_code = get_unix(splitted_line[0],splitted_line[1])

class Strategy:
    def __init__(self) -> None:
      pass
    def Von_Neuman(self, prices, cant_part,  tres_hold) -> int :  #tick_list: List[Any] , unix_limit_1: float, unix_limit_2: float , bid_or_ask: str,cant_part: int, tres_hold: float) -> Tuple:
      up_down= []
      #CAMBIO PARA AJUSTARLO AL LARGO DE LA LISTA DE VALORES NUMERICOS UTILES
      if cant_part<len(prices):
          for index_groups in range(cant_part-1):
              try:
                  if prices[index_groups*10]<prices[(index_groups+1)*10]:
                      up_down.append(1)
                  elif (prices[index_groups*10]>prices[(index_groups+1)*10]):
                      up_down.append(-1)
                  else:
                      up_down.append(0)
              except IndexError:
                  up_down.append(0)
          try:
              avg = sum(up_down)/len(up_down)
              if (avg>=-tres_hold) and (avg<=tres_hold):
                  return 0
              elif avg>0 :
                  return -1
              else:
                  return 1
          except ZeroDivisionError:
              return 2
      else:
          return 0
def period_action(p, Tick_list, path):
    aux = []
    with open(path,'a') as file_for_executing:
      for index, value in enumerate(Tick_list):
          if int(value.time[3:5])%p == 0  and not(value.time[:2]=="00" and value.time[3:5] == "00") and not(value.time[:5] in aux):
            file_for_executing.writelines(f"{value.unix_code}, {value.time[:5]}, {value.bid}, {value.ask} \n")
            aux.append(value.time[:5])
      return path

Tick_list = []
with open ('31-05-2023.txt','r') as file:
  for line in file:
    Tick_list.append(Tick(line))

path_2 = 'execution_time.txt'
################################################3
action_p = period_action(15, Tick_list, path_2)#3
################################################3

def commit_by_sell(Tick_list, path_2):
  strat = Strategy()
  path_3 = 'buy_sell_committed.txt'
  id = 0
  with open (path_3,'a') as bs :
    with open (path_2,'r') as file_2:
      file_2 = file_2.readlines()
      ext_file_2 = []
      for value in file_2 :
        ext_file_2.append(value.split(","))
    before = 0
    for value_1 in ext_file_2:
      prices = []
      for value_2 in Tick_list:
        if value_2.unix_code < float(value_1[0]) and float(value_2.unix_code) > before:
          prices.append(value_2.bid)
      before = float(value_1[0])

      buy_sell = strat.Von_Neuman(prices, 4, 0.3)
      if buy_sell == 1 and id != 0:
          bs.writelines(f"{id}, sell, {value_1[2]}, {value_1[3]}")
          id += 1
      if buy_sell == -1:
          bs.writelines(f"{id}, buy, {value_1[2]}, {value_1[3]}")
          id += 1
  return path_3
path_3 = commit_by_sell(Tick_list, path_2)

def reports(path_3):
  path_4 = "reports_PL.txt"
  with open ('buy_sell_committed.txt','r') as file_1:
    file_1 =file_1.readlines()
    with open(path_4,'a') as written_file :
      for index_11, value_11  in enumerate(file_1):
        P_L=0
        splitted_line_0 =  value_11.split(",")
        end_bid = float(splitted_line_0[2])
        end_ask = float(splitted_line_0[3])
        for index_1, value_1 in enumerate(file_1) :
            if (int(splitted_line_0[0])>0) :
                  splitted_line = value_1.split(",")
                  if len(splitted_line)>0 :
                    decision= (splitted_line[1])
                    entry_bid = float(splitted_line[2])
                    entry_ask = float(splitted_line[3])
                    profit = 0
                    if decision.strip() == "buy":
                        profit = entry_bid - end_ask
                    elif decision.strip() == "sell":
                        profit = end_bid - entry_ask
                    P_L += profit
        if splitted_line_0[0].strip() == "0":
            splitted_line = file_1[0].split(",")
            if splitted_line[1].strip() == "buy":
                written_file.writelines(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}, -{splitted_line_0[2]}\n")
            else:
                written_file.writelines(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}\n")
        else:
            written_file.writelines(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}, {str(P_L)[:8]}\n")
  return path_4
path_4 = reports(path_3)


  1.07030 

0,  buy,   1.07020,   1.07030 


written


### O U T _ HELP


In [4]:
def commit_by_sell(Tick_list, path_2):
  strat = Strategy()
  path_3 = 'buy_sell_committed.txt'
  id = 0
  with open (path_3,'a') as bs :
    with open (path_2,'r') as file_2:
      file_2 = file_2.readlines()
      ext_file_2 = []
      for value in file_2 :
        ext_file_2.append(value.split(","))
    before = 0
    for value_1 in ext_file_2:
      prices = []
      for value_2 in Tick_list:
        if value_2.unix_code < float(value_1[0]) and float(value_2.unix_code) > before:
          prices.append(value_2.bid)
      before = float(value_1[0])

      buy_sell = strat.Von_Neuman(prices, 4, 0.3)
      if buy_sell == 1 and id != 0:
          bs.writelines(f"{id}, sell, {value_1[2]}, {value_1[3]}")
          id += 1
      if buy_sell == -1:
          bs.writelines(f"{id}, buy, {value_1[2]}, {value_1[3]}")
          id += 1
  return path_3
commit_by_sell(Tick_list, path_2)

def reports(path_3):
  path_4 = "reports_PL.txt"
  with open ('buy_sell_committed.txt','r') as file_1:
    file_1 =file_1.readlines()
    with open(path_4,'a') as written_file :
      for index_11, value_11  in enumerate(file_1):
        P_L=0
        splitted_line_0 =  value_11.split(",")
        end_bid = float(splitted_line_0[2])
        end_ask = float(splitted_line_0[3])
        for index_1, value_1 in enumerate(file_1) :
            if (int(splitted_line_0[0])>0) :
                  splitted_line = value_1.split(",")
                  if len(splitted_line)>0 :
                    decision= (splitted_line[1])
                    entry_bid = float(splitted_line[2])
                    entry_ask = float(splitted_line[3])
                    profit = 0
                    if decision.strip() == "buy":
                        profit = entry_bid - end_ask
                    elif decision.strip() == "sell":
                        profit = end_bid - entry_ask
                    P_L += profit
        if splitted_line_0[0].strip() == "0":
            splitted_line = file_1[0].split(",")
            if splitted_line[1].strip() == "buy":
                print(splitted_line_0[3])
                written_file.writelines(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}, -{splitted_line_0[2]}\n")
                print(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}\n")
                print("written")
            else:
                written_file.writelines(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}\n")
        else:
            written_file.writelines(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}, {str(P_L)[:8]}\n")
  return path_4

In [ ]:
def reports(path_2):
  path_4 = "reports_PL.txt"
  with open (path_2,'r') as file_2:
      file_2 = file_2.readlines()
      ext_file_2 = []
      for value in file_2 :
        ext_file_2.append(value.split(","))
  with open (path_4,'a') as bs :
    with open (path_1,'r') as file_1:
      for value_1 in file_1 :







In [43]:
def reports(path_3):
  with open ('buy_sell_committed.txt','r') as file_1:
    file_1 =file_1.readlines()
    with open('reports_PL.txt','a') as written_file :
      for index_11, value_11  in enumerate(file_1):
        P_L=0
        splitted_line_0 =  value_11.split(",")
        end_bid = float(splitted_line_0[2])
        end_ask = float(splitted_line_0[3])
        for index_1, value_1 in enumerate(file_1) :
            if (int(splitted_line_0[0])>0) :
                  splitted_line = value_1.split(",")
                  if len(splitted_line)>0 :
                    decision= (splitted_line[1])
                    entry_bid = float(splitted_line[2])
                    entry_ask = float(splitted_line[3])
                    profit = 0
                    if decision.strip() == "buy":
                        profit = entry_bid - end_ask
                    elif decision.strip() == "sell":
                        profit = end_bid - entry_ask
                    P_L += profit
        if splitted_line_0[0].strip() == "0":
            splitted_line = file_1[0].split(",")
            if splitted_line[1].strip() == "buy":
                print(splitted_line_0[3])
                written_file.writelines(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}, -{splitted_line_0[2]}\n")
                print(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}\n")
                print("written")
            else:
                written_file.writelines(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}\n")
        else:
            written_file.writelines(f"{splitted_line_0[0]}, {splitted_line_0[1]}, {splitted_line_0[2]}, {splitted_line_0[3]}, {str(P_L)[:8]}\n")



### SECOND HELP